In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

## Importing dataset

In [13]:
from dgl.data import FraudAmazonDataset

dataset = FraudAmazonDataset()

g_raw = dataset[0]
train_nid = torch.nonzero(g_raw.ndata['train_mask'], as_tuple=True)[0]
# convert to homogeneous graph
g_homo = dgl.to_homogeneous(g_raw)
# build train graph
g_train = g_homo.subgraph(train_nid)

print(g_homo.ndata)


Done loading data from cached files.
{'_ID': tensor([    0,     1,     2,  ..., 11941, 11942, 11943]), '_TYPE': tensor([0, 0, 0,  ..., 0, 0, 0])}


## Creating the model

In [3]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

## Training the model

In [14]:
def train(g, g_raw, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g_raw.ndata['feature']
    labels = g_raw.ndata['label']
    train_mask = g_raw.ndata['train_mask']
    val_mask = g_raw.ndata['val_mask']
    test_mask = g_raw.ndata['test_mask']

    train_acc_record = []
    val_acc_record = []
    test_acc_record = []
    
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        train_acc_record.append(train_acc)
        val_acc_record.append(val_acc)
        test_acc_record.append(test_acc)

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
    
    return train_acc_record, val_acc_record, test_acc_record
    

model = GCN(g_raw.ndata['feature'].shape[1], 16, dataset.num_classes)
train_acc, val_acc, test_acc = train(g, g_raw, model)

/tmp/ipykernel_57061/2926700447.py:25: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  loss = F.cross_entropy(logits[train_mask], labels[train_mask])
/tmp/ipykernel_57061/2926700447.py:28: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
/tmp/ipykernel_57061/2926700447.py:29: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
/tmp/ipykernel_57061/2926700447.py:30: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally a

In epoch 0, loss: 100.124, val acc: 0.075 (best 0.075), test acc: 0.106 (best 0.106)
In epoch 5, loss: 1.471, val acc: 0.925 (best 0.925), test acc: 0.894 (best 0.894)
In epoch 10, loss: 4.490, val acc: 0.925 (best 0.925), test acc: 0.894 (best 0.894)
In epoch 15, loss: 5.864, val acc: 0.925 (best 0.925), test acc: 0.894 (best 0.894)
In epoch 20, loss: 6.339, val acc: 0.925 (best 0.925), test acc: 0.894 (best 0.894)
In epoch 25, loss: 6.276, val acc: 0.925 (best 0.925), test acc: 0.894 (best 0.894)
In epoch 30, loss: 5.887, val acc: 0.925 (best 0.925), test acc: 0.894 (best 0.894)
In epoch 35, loss: 5.307, val acc: 0.925 (best 0.925), test acc: 0.894 (best 0.894)
In epoch 40, loss: 4.619, val acc: 0.925 (best 0.925), test acc: 0.894 (best 0.894)
In epoch 45, loss: 3.873, val acc: 0.925 (best 0.925), test acc: 0.894 (best 0.894)
In epoch 50, loss: 3.098, val acc: 0.925 (best 0.925), test acc: 0.894 (best 0.894)
In epoch 55, loss: 2.309, val acc: 0.925 (best 0.925), test acc: 0.894 (best